In [1]:
import datetime
import time
import json
import psycopg2

# url = "hdfs://127.0.0.1:9000/socmed_input/"
# out_url = "hdfs://127.0.0.1:9000/socmed_output/"

In [2]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])
def process_rdd(rdd):
    # Collect the tuples inside the RDD as a list
    data = rdd.collect()
    # Process the data
    for row in data:
        insertToSQL(row)
        

def insertToSQL(data):
    
    conn = psycopg2.connect(database="bigdatatest",
     host="localhost",
     user="geraldakbar",
     password='bigdata'
     )

    cur = conn.cursor()
    
    social_media = data[0]
    timestamp = data[1]
    count = data[2]
    uniquecount = data[3]

    query = f"""INSERT INTO social_media_data (social_media, timestamp, count, uniquecount)
                    VALUES ('{social_media}', '{timestamp}', {count}, {uniquecount})
                    ON CONFLICT (social_media, timestamp) DO UPDATE
                    SET count = CASE WHEN social_media_data.count IS NULL THEN 1 ELSE social_media_data.count + {count} END,
                    uniquecount = CASE WHEN social_media_data.uniquecount IS NULL THEN 1 ELSE social_media_data.uniquecount + {uniquecount} END,
                    updated_at = NOW();"""


    cur.execute(query)
    print("Inserted Data:",data)
    conn.commit()
    cur.close()
    conn.close()

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, ['facebook'],
                                      {"metadata.broker.list": "localhost:9092"})

def getFB(lines):
    def map(line):
        result = []
        postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
#         comments = line["comments"]["data"]
        result.append((("facebook", postDate), 1))
#         for comment in comments:
#             date = comment["created_time"][0:10] + ' ' + normalizeTime(comment["created_time"][11:19])
#             result.append((("facebook", date), 1))
        return result

    def userMap(line):
        user = line["from"]["name"]
        postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
        return (("facebook", postDate, user), 1)

    lines = lines.window(5, 5)
    lines = lines.map(lambda x: json.loads(x[1]))
#     cursor = conn.cursor()
    fbPost = lines.flatMap(map)
    fbPost = fbPost.reduceByKey(lambda x, y: x + y)

    user = lines.map(userMap)
    user = user.transform(lambda rdd: rdd.distinct())
    user = user.map(lambda x: ((x[0][0], x[0][1]), x[1]))
    user = user.reduceByKey(lambda x, y: x + y)

    fbPost = fbPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
    fbPost = fbPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
    # fbPost = masukin ke postgre
    return fbPost

result = getFB(lines)
# Print
result.foreachRDD(process_rdd)
ssc.start()
ssc.awaitTermination()
# test = fbPost.take(5)
# insertToSQL(test,conn,cur)

Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 16:40:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:05:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 16:55:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 16:55:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
Inserted Data: ('facebook', '2023-04-04 17:05:00', 1, 1)


KeyboardInterrupt: 

In [5]:
spark

Inserted Data: ('facebook', '2023-04-04 17:10:00', 1, 1)
